# Core Statistics Using Python
### Hana Choi, Simon Business School, University of Rochester


# Linear Probability Model

## Topics covered

- Binary outcomes: The Linear Probability Model (LPM)
- Probit and logit models (for references only) 

## Required packages

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

## Load data: titanic2.csv

In [ ]:
# Load data
titanic2 = pd.read_csv("/Users/hanachoi/Dropbox/teaching/core_statistics/Data/titanic2.csv")

# Display the first few rows of the data
print(titanic2.head())
print('----')

# Summary statistics
print(titanic2.describe())

# Linear Probability Model (LPM)

## Simple LPM with one regressor

In [ ]:
lpm_simple = smf.ols('Survived ~ Fare', data=titanic2).fit()
print(lpm_simple.summary().tables[1])

# Plotting
plt.scatter(titanic2['Fare'], titanic2['Survived'], alpha=0.5)
plt.plot(titanic2['Fare'], lpm_simple.predict(), color='red')
plt.xlabel('Fare')
plt.ylabel('Survived')
plt.show()

## LPM with four regressors

### Regression model

In [ ]:
# LPM with four regressors
lpm_model = smf.ols('Survived ~ Gender + Class + Age + Fare', data=titanic2).fit()
print(lpm_model.summary().tables[1])

### 99% Confidence intervals for $\beta_j$

In [ ]:
# Confidence intervals 
conf_int = lpm_model.conf_int(alpha=0.01)
print(conf_int)

### Predictions

We will make predictions for three different cases.

1. Predict survival probability for a 30y male in 3rd class paying \$30

    $1.35-0.503-0.205*3-0.005*30-0.0005*30 = 0.067$ (some rounding error)
<br>

2. Predict survival probability for a 30y female in 3rd class paying \$30

    $1.35-0.503*0-0.205*3-0.005*30-0.0005*30 = 0.57$ (some rounding error)
<br>

3. Predicted probability for a 2y female in 1st class paying \$200

    $1.35-0.503*0-0.205*1-0.005*2-0.0005*200 = 1.03$ (Probability $>1$)

In [ ]:
# Create a dataframe with values at which we want to make predictions
new_data = pd.DataFrame({
    'Gender': [1, 0, 0], 
    'Class': [3, 3, 1], 
    'Age': [30, 30, 2], 
    'Fare': [30, 30, 200]
})

# Get predictions at those values
predictions = lpm_model.get_prediction(new_data)
print(predictions.summary_frame(alpha=0.05))

### Visualization

- Recall that the LPM can produce predictions outside the (0,1) interval
- Let's see how often it happens with this data 
- Obtain predicted values and and graph them

In [ ]:
# Predicting survival probabilities for each observation in the dataset:
surv_prob = lpm_model.predict()

# Plot histogram of the predicted survival probabilities
plt.hist(surv_prob, bins=20, color='grey', edgecolor='black', alpha=0.7)
plt.title('Histogram of Predicted Survival Probabilities')
plt.xlabel('Predicted Survival Probability')
plt.ylabel('Frequency')
plt.show()

# There are indeed some problematic cases

## Heteroskedasticity-Robust Standard Errors

- LPM models should always report HR SEs (since these models are heteroskedastic by definition) 
- Obtain HR Standard Errors for the model with 4 X's (other cases are similar)

In [ ]:
lpm_model_HRse = lpm_model.get_robustcov_results(cov_type='HC1')
print(lpm_model_HRse.summary().tables[1])
print('----')

predictions_HRse = lpm_model_HRse.get_prediction(new_data)
print(predictions_HRse.summary_frame(alpha=0.05))

## LPM with all 6 regressors

### Regression model

In [ ]:
# Create formula
dependent_var = 'Survived'
independent_vars = titanic2.columns.drop(dependent_var)
formula_all = f"{dependent_var} ~ " + " + ".join(independent_vars)

# LPM with all 6 regressors available in the dataset and report HR SEs
lpm_model_all = smf.ols(formula_all, data=titanic2).fit(cov_type='HC1')
print(lpm_model_all.summary().tables[1])

# Probit and Logit Models (For Reference Only)

## Probit model

### Running Probit model

In [ ]:
# Estimating Probit model with four regressors
probit_model = smf.probit('Survived ~ Gender + Class + Age + Fare', data=titanic2).fit()
print(probit_model.summary())

### Computing marginal effects

- Check the documentation and available options: https://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.ProbitResults.get_margeff.html

In [ ]:
marginal_effects_probit = probit_model.get_margeff() 
print(marginal_effects_probit.summary())

### Predict survival probabilities using Probit model and graph them

In [ ]:
probit_predictions = probit_model.predict()
plt.hist(probit_predictions, bins=20, alpha=0.5, label='Probit')
plt.legend()
plt.show()

# All between 0 and 1 now!

## Logit model

### Running Logit model

In [ ]:
# Estimating Logit model with four regressors
logit_model = smf.logit('Survived ~ Gender + Class + Age + Fare', data=titanic2).fit()
print(logit_model.summary())

### Computing marginal effects

In [ ]:
marginal_effects_logit = logit_model.get_margeff() 
print(marginal_effects_logit.summary())

### Predict survival probabilities using Logit model and graph them

In [ ]:
logit_predictions = logit_model.predict()
plt.hist(logit_predictions, bins=20, alpha=0.5, label='Logit', color='green')
plt.legend()
plt.show()

# All between 0 and 1 again!